In [1]:
import pandas as pd
#import pandas.io.sql as sqlio
#import sqlalchemy as sql
import psycopg2
#import matplotlib.pyplot as plt
import numpy as np
import os

#from dotenv import load_dotenv
#from matplotlib import colors
#from matplotlib.ticker import PercentFormatter
   
from sqlalchemy import create_engine

# load up the variables from the .env file.
#load_dotenv()

In [2]:
# File to Load (Remember to Change These)
file_to_load = "Resources/StatetoAbbrev.csv"
states_data = pd.read_csv(file_to_load)
states_data.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [3]:
states_data['state id']=states_data.index+1
states_data.head()

,State,Abbrev,Code,state id
0,Alabama,Ala.,AL,1
1,Alaska,Alaska,AK,2
2,Arizona,Ariz.,AZ,3
3,Arkansas,Ark.,AR,4
4,California,Calif.,CA,5


In [4]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Coal Production-StateRankings.csv"
Coal_Production = pd.read_csv(file_to_load)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",Note: Rankings are based on the full source data values.
0,1,WY,304188,NaN
1,2,WV,95365,NaN
2,3,PA,49883,NaN
3,4,IL,49563,NaN
4,5,KY,39567,NaN


In [13]:
Coal_Production.drop(columns=["Note: Rankings are based on the full source data values."],inplace=True)
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",State id
0,1,WY,304188,1
1,2,WV,95365,2
2,3,PA,49883,3
3,4,IL,49563,4
4,5,KY,39567,5


In [14]:
Coal_Production['State id']=Coal_Production.index+1
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",State id
0,1,WY,304188,1
1,2,WV,95365,2
2,3,PA,49883,3
3,4,IL,49563,4
4,5,KY,39567,5


In [18]:
Coal_Production['State id']=Coal_Production.index+1
Coal_Production.head()

,Rank,State,"Coal Production, thousand short tons",State id
0,1,WY,304188,1
1,2,WV,95365,2
2,3,PA,49883,3
3,4,IL,49563,4
4,5,KY,39567,5


In [15]:
# File to Load (Remember to Change These)
file_to_load = "Resources/Crude Oil Production-StateRankings.csv"
CrudeOil_Production = pd.read_csv(file_to_load)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day",Note: Rankings are based on the full source data values. Excludes federal offshore production.
0,1,TX,4395,NaN
1,2,NM,885,NaN
2,3,ND,862,NaN
3,4,CO,471,NaN
4,5,AK,404,NaN


In [16]:

CrudeOil_Production.drop(columns=["Note: Rankings are based on the full source data values. Excludes federal offshore production."],inplace=True)
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day"
0,1,TX,4395
1,2,NM,885
2,3,ND,862
3,4,CO,471
4,5,AK,404


In [17]:
CrudeOil_Production['State id']=CrudeOil_Production.index+1
CrudeOil_Production.head()

,Rank,State,"Crude Oil Production, thousand barrels per day",State id
0,1,TX,4395,1
1,2,NM,885,2
2,3,ND,862,3
3,4,CO,471,4
4,5,AK,404,5


In [ ]:
# File to Load (Remember to Change These)
file_to_load = "Resources/usretechnicalpotential.csv"


# Read Renewable energy potential file and store into Pandas data frame
renewable_energy_data = pd.read_csv(file_to_load)
renewable_energy_data.columns

### Cleaning data

In [ ]:
renewable_energy_data.rename(columns={'Unnamed: 0' : 'State'}, inplace=True)
bio_power_potential = renewable_energy_data[['State', 'biopowerSolid_GWh', 'biopowerGaseous_GWh']]
bio_power_potential.head()

In [ ]:
# set the variables for the pull from SQL.  username and password are in an .env file.  You will need those to make this run
# on your machine.

host = 'localhost'
port = 5432
dbname='ETL_Project'
username = os.environ.get('USER_NAME')
pwd = os.environ.get('PASSWORD')


In [ ]:
# set up the connection to the database "ETL_Project"
connection_string = f'{username}:{pwd}@{host}:{port}/{dbname}'
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

In [ ]:
# set up the connection to the database "ETL_Project"
"""
conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))
sql = "SQL statement"
dat = sqlio.read_sql_query(sql, conn)"""
